<a href="https://colab.research.google.com/github/srinath-96/Dashboard_generator/blob/main/dash_videogamesales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash plotly pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [22]:
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Load the dataset
df = pd.read_csv('/content/vgchartz-2024.csv')

# Clean the data
df = df.dropna(subset=['total_sales'])
df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce')

# Filter for Sony consoles
sony_consoles = ['PS2', 'PS3', 'PS4']
sony_data = df[df['console'].isin(sony_consoles)]

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)

In [23]:


# Define the layout
app.layout = html.Div([
    html.H1('Sony PlayStation Video Game Sales Dashboard', style={'textAlign': 'center', 'color': '#003791'}),

    html.Div([
        html.Div([
            html.Label('Select Console:'),
            dcc.Dropdown(
                id='console-dropdown',
                options=[{'label': console, 'value': console} for console in sony_consoles],
                value=sony_consoles,
                multi=True
            )
        ], style={'width': '30%', 'display': 'inline-block'}),

        html.Div([
            html.Label('Select Genre:'),
            dcc.Dropdown(
                id='genre-dropdown',
                options=[{'label': genre, 'value': genre} for genre in sorted(sony_data['genre'].unique())],
                value=[],
                multi=True
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'}),

        html.Div([
            html.Label('Sales Range (millions):'),
            dcc.RangeSlider(
                id='sales-slider',
                min=0,
                max=round(sony_data['total_sales'].max()),
                value=[0, round(sony_data['total_sales'].max())],
                marks={i: f'{i}M' for i in range(0, round(sony_data['total_sales'].max()) + 5, 5)},
                step=1
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '5%'})
    ], style={'marginBottom': '20px'}),

    html.Div([
        html.Div([
            html.H2('Sales by Console', style={'textAlign': 'center'}),
            dcc.Graph(id='sales-by-console')
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.H2('Top Publishers', style={'textAlign': 'center'}),
            dcc.Graph(id='top-publishers')
        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ]),

    html.Div([
        html.Div([
            html.H2('Regional Sales Distribution', style={'textAlign': 'center'}),
            dcc.Graph(id='regional-sales')
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.H2('Critic Scores vs. Sales', style={'textAlign': 'center'}),
            dcc.Graph(id='critic-scores')
        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ]),

    html.Div([
        html.H2('Best-Selling Games by Publisher', style={'textAlign': 'center'}),
        html.Div(id='best-selling-games')
    ])
])

# Callback for updating all visualizations
@app.callback(
    [
        Output('sales-by-console', 'figure'),
        Output('top-publishers', 'figure'),
        Output('regional-sales', 'figure'),
        Output('critic-scores', 'figure'),
        Output('best-selling-games', 'children')
    ],
    [
        Input('console-dropdown', 'value'),
        Input('genre-dropdown', 'value'),
        Input('sales-slider', 'value')
    ]
)
def update_dashboard(selected_consoles, selected_genres, sales_range):
    # Filter data based on selections
    filtered_data = sony_data[sony_data['console'].isin(selected_consoles)]

    if selected_genres:
        filtered_data = filtered_data[filtered_data['genre'].isin(selected_genres)]

    filtered_data = filtered_data[(filtered_data['total_sales'] >= sales_range[0]) &
                                 (filtered_data['total_sales'] <= sales_range[1])]

    # Sales by Console
    console_sales = filtered_data.groupby('console')['total_sales'].sum().reset_index()
    sales_fig = px.bar(
        console_sales,
        x='console',
        y='total_sales',
        title='Total Sales by Console (millions)',
        color='console',
        color_discrete_map={'PS2': '#003791', 'PS3': '#00439C', 'PS4': '#0055A7'}
    )
    sales_fig.update_layout(xaxis_title='Console', yaxis_title='Total Sales (millions)')

    # Top Publishers
    publisher_sales = filtered_data.groupby('publisher')['total_sales'].sum().reset_index()
    top_publishers = publisher_sales.sort_values('total_sales', ascending=False).head(10)
    publishers_fig = px.bar(
        top_publishers,
        x='publisher',
        y='total_sales',
        title='Top 10 Publishers by Sales (millions)'
    )
    publishers_fig.update_layout(xaxis_title='Publisher', yaxis_title='Total Sales (millions)')

    # Regional Sales
    regional_data = pd.DataFrame({
        'Region': ['North America', 'Japan', 'Europe/PAL', 'Other'],
        'Sales': [
            filtered_data['na_sales'].sum(),
            filtered_data['jp_sales'].sum(),
            filtered_data['pal_sales'].sum(),
            filtered_data['other_sales'].sum()
        ]
    })
    regional_fig = px.pie(
        regional_data,
        values='Sales',
        names='Region',
        title='Regional Sales Distribution',
        color_discrete_sequence=px.colors.sequential.Blues_r
    )

    # Critic Scores vs. Sales
    critic_data = filtered_data.dropna(subset=['critic_score'])
    critic_fig = px.scatter(
        critic_data,
        x='critic_score',
        y='total_sales',
        color='console',
        size='total_sales',
        hover_data=['title', 'publisher', 'release_date'],
        title='Critic Scores vs. Total Sales',
        color_discrete_map={'PS2': '#003791', 'PS3': '#00439C', 'PS4': '#0055A7'}
    )
    critic_fig.update_layout(xaxis_title='Critic Score', yaxis_title='Total Sales (millions)')

    # Best-Selling Games by Publisher
    top_publishers_list = top_publishers['publisher'].tolist()
    best_games = []

    for publisher in top_publishers_list:
        publisher_games = filtered_data[filtered_data['publisher'] == publisher]
        if not publisher_games.empty:
            best_game = publisher_games.loc[publisher_games['total_sales'].idxmax()]
            best_games.append({
                'Publisher': publisher,
                'Game': best_game['title'],
                'Console': best_game['console'],
                'Sales (millions)': round(best_game['total_sales'], 2),
                'Critic Score': best_game['critic_score']
            })

    best_games_table = dash_table.DataTable(
        id='best-games-table',
        columns=[{'name': col, 'id': col} for col in ['Publisher', 'Game', 'Console', 'Sales (millions)', 'Critic Score']],
        data=best_games,
        style_cell={'textAlign': 'left', 'padding': '10px'},
        style_header={'backgroundColor': '#003791', 'color': 'white', 'fontWeight': 'bold'},
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': '#f2f2f2'
            }
        ]
    )

    return sales_fig, publishers_fig, regional_fig, critic_fig, best_games_table




In [24]:
app.run(debug=True)

<IPython.core.display.Javascript object>